In [9]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

college = pd.read_csv("csv.zip")
college

,University,Department,Overall Pay,LW Ratio,Academic Year,Program Year,12 M Gross Pay,9 M Gross Pay,3 M Gross Pay,Fees,Comments
0,Cornell University (CU),Sociology,"$34,605",1.43,2020-2021,1st,NaN,"$28,654","$6,037",$86,NaN
1,Oklahoma State University (OSU),Counseling psychology,"$2,855",0.15,2020-2021,1st,NaN,"$9,855",NaN,"$7,000",There fees are extremely high. They give you h...
2,Emory University,Biological and Biomedical Sciences,"$32,569",1.40,2021-2022,1st,"$32,569",NaN,NaN,NaN,NaN
3,Boston University (BU),PIBS,"$36,782",1.28,2021-2022,1st,"$36,782",NaN,NaN,NaN,NaN
4,Brown University,Computer Science,"$41,930",1.84,2021-2022,1st,NaN,"$29,014","$12,916",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9619,Duke University (DU),Biochemistry,"$29,840",1.59,2012-2013,6th and up,"$29,840",NaN,NaN,NaN,NaN
9620,University of Virginia (UVA),Chemical Engineering,"$32,000",1.63,2013-2014,4th,"$32,000",NaN,NaN,NaN,"This includes health insurance coverage, tuiti..."
9621,Duke University (DU),Biomedical Engineering,"$32,000",1.70,2013-2014,2nd,"$32,000",NaN,NaN,NaN,This is the stipend associated with the NSF Fe...
9622,University of Virginia (UVA),Chemical Engineering,"$25,000",1.28,2013-2014,4th,"$25,000",NaN,NaN,NaN,This stipend includes full health insurance co...


In [15]:
college = college.drop(columns = ['Comments', 'Academic Year', 'Fees', 'Program Year'])

In [14]:
college.sort_values(by = 'LW Ratio')

,University,Department,Overall Pay,LW Ratio,Academic Year,Program Year,12 M Gross Pay,9 M Gross Pay,3 M Gross Pay,Fees,Comments
4267,University of California - San Diego (UCSD),Information Technology,"$-900,000",-34.01,2018-2019,2nd,"$100,000",NaN,NaN,"$1,000,000",NaN
3140,Purdue University - West Lafayette,physics,"$-80,000",-3.78,2019-2020,1st,"$20,000",NaN,NaN,"$100,000",NaN
592,Massachusetts Institute of Technology (MIT),Chemical,"$-100,000",-3.58,2022-2023,6th and up,NaN,NaN,NaN,"$100,000",NaN
593,Massachusetts Institute of Technology (MIT),Chemical,"$-100,000",-3.58,2022-2023,6th and up,NaN,NaN,NaN,"$100,000",NaN
8771,Massachusetts Institute of Technology (MIT),Environmental,"$-73,344",-2.63,2015-2016,1st,"$5,622",NaN,NaN,"$78,966",NaN
...,...,...,...,...,...,...,...,...,...,...,...
9466,University of South Florida,Curriculum and Instruction,"$12,300",NaN,2004-2005,1st,NaN,"$10,000","$2,500",$200,NaN
9505,NaN,Electrical and Computer Engineering,"$24,337",NaN,2014-2015,4th,"$28,000",NaN,NaN,"$3,663",NaN
9528,Pardee RAND Graduate School,Policy Analysis,"$22,225",NaN,2014-2015,1st,"$22,225",NaN,NaN,NaN,NaN
9532,The Scripps Research Institute,NaN,"$30,000",NaN,2013-2014,5th,"$30,000",NaN,NaN,NaN,NaN
